# Algorithmic Competition Analysis

This notebook analyzes simulation results: compares policies, tests coordination, and visualizes welfare effects.


In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import sys

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent / "backend"))

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)


## Load Aggregated Results


In [ ]:
# Load aggregated metrics
metrics_path = Path("../data/aggregates/metrics.csv")
if metrics_path.exists():
    df = pd.read_csv(metrics_path)
    print(f"Loaded {len(df)} runs")
    print(df.head())
else:
    print("No metrics.csv found. Run: python scripts/aggregate_runs.py")
    df = pd.DataFrame()


## Compare Policies: Statistical Tests


In [ ]:
if len(df) > 0:
    # Group by policy
    policy_stats = df.groupby("policy")[["mean_price", "total_welfare", "consumer_surplus_per_consumer"]].agg(["mean", "std", "count"]).round(4)
    print("Policy Comparison:")
    print(policy_stats)
    
    # T-test: myopic vs bandit
    if "myopic" in df["policy"].values and "epsilon_bandit" in df["policy"].values:
        myopic = df[df["policy"] == "myopic"]["mean_price"]
        bandit = df[df["policy"] == "epsilon_bandit"]["mean_price"]
        t_stat, p_val = stats.ttest_ind(myopic, bandit)
        print(f"\nT-test (myopic vs bandit, mean_price): t={t_stat:.4f}, p={p_val:.4f}")
        
        # Confidence intervals
        def ci(data, conf=0.95):
            n = len(data)
            m = data.mean()
            se = data.std() / np.sqrt(n)
            t_crit = stats.t.ppf(1 - (1-conf)/2, df=n-1) if n > 1 else 1.96
            return m - t_crit*se, m + t_crit*se
        
        ci_myopic = ci(myopic)
        ci_bandit = ci(bandit)
        print(f"\nMyopic mean_price: {myopic.mean():.4f} [{ci_myopic[0]:.4f}, {ci_myopic[1]:.4f}]")
        print(f"Bandit mean_price: {bandit.mean():.4f} [{ci_bandit[0]:.4f}, {ci_bandit[1]:.4f}]")


## Visualize Welfare Comparison


In [ ]:
if len(df) > 0 and "policy" in df.columns:
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Mean price
    sns.boxplot(data=df, x="policy", y="mean_price", ax=axes[0])
    axes[0].set_title("Mean Price by Policy")
    axes[0].tick_params(axis="x", rotation=45)
    
    # Consumer surplus
    sns.boxplot(data=df, x="policy", y="consumer_surplus_per_consumer", ax=axes[1])
    axes[1].set_title("Consumer Surplus by Policy")
    axes[1].tick_params(axis="x", rotation=45)
    
    # Total welfare
    sns.boxplot(data=df, x="policy", y="total_welfare", ax=axes[2])
    axes[2].set_title("Total Welfare by Policy")
    axes[2].tick_params(axis="x", rotation=45)
    
    plt.tight_layout()
    plt.show()


## Coordination Metrics


In [ ]:
if len(df) > 0 and "cross_corr_lag1_f0_f1" in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Cross-correlation
    sns.boxplot(data=df, x="policy", y="cross_corr_lag1_f0_f1", ax=axes[0])
    axes[0].axhline(y=0, color="r", linestyle="--", alpha=0.5)
    axes[0].set_title("Price Cross-Correlation (Lag 1)")
    axes[0].tick_params(axis="x", rotation=45)
    
    # Identical price share
    sns.boxplot(data=df, x="policy", y="identical_price_share", ax=axes[1])
    axes[1].set_title("Identical Price Share")
    axes[1].tick_params(axis="x", rotation=45)
    
    plt.tight_layout()
    plt.show()
